In [13]:
import pandas as pd
import spacy
import requests
import ast
import deepl 

In [29]:
translator = deepl.Translator("6780cef2-4c9d-787f-4999-9649bd278538:fx") 

In [14]:
nlp1 = spacy.load("de_core_news_sm")  # Model 1
nlp2 = spacy.load('de_core_news_md')  # Model 2
nlp3 = spacy.load('de_core_news_lg')  # Model 3

In [15]:
df_german = pd.read_csv('../scraping/data/extractor_all_articles_20minuten.csv')
df_french = pd.read_csv('../scraping/data/extractor_all_articles_20minutes.csv')
df_italian = pd.read_csv('../scraping/data/extractor_all_articles_20minuti.csv')

In [ ]:
dict_all_countries = pd.read_csv('dicts/allCountries.csv')
known_locations = set(dict_all_countries['LocationName'])

In [17]:
dict_all_countries['BiggerEntity'] = dict_all_countries['CountryName'].str.split('/').str[-1]

In [34]:
unique_location_names = dict_all_countries['BiggerEntity'].unique()

In [39]:
def translate_to_german(text):
    try:
        result = translator.translate_text(text, target_lang="DE")
        return result.text
    except Exception as e:
        print(f"An error occurred during translation: {e}")
        return None  # Return None if translation fails

In [44]:
translated_names = {name: translate_to_german(name) for name in unique_location_names}

An error occurred during translation: text parameter must be a string or an iterable of strings


In [46]:
name_translation_mapping = {original: translated_names[original] for original in unique_location_names}

In [47]:
dict_all_countries['BiggerEntity_German'] = dict_all_countries['BiggerEntity'].map(name_translation_mapping)


In [50]:
new_locations = set(dict_all_countries['BiggerEntity_German'].unique())

In [52]:
known_locations.update(new_locations)

In [56]:
def extract_locations_with_voting_system(text, known_locations):
    # Process the text with all three models
    doc1 = nlp1(text)
    doc2 = nlp2(text)
    doc3 = nlp3(text)
    
    # Extract locations from each model
    locations1 = {ent.text for ent in doc1.ents if ent.label_ == 'LOC'}
    locations2 = {ent.text for ent in doc2.ents if ent.label_ == 'LOC'}
    locations3 = {ent.text for ent in doc3.ents if ent.label_ == 'LOC'}
    
    # Perform a majority vote
    all_locations = locations1 | locations2 | locations3
    final_locations = {loc for loc in all_locations if sum([loc in locations for locations in [locations1, locations2, locations3]]) >= 2}
    
    # Filter final locations to include only those in the known locations
    final_known_locations = {loc for loc in final_locations if loc in known_locations}
    
    return final_known_locations


In [58]:
def merge_similar_entities(entities):
    merged_entities = set()
    for entity in entities:
        if not any(e for e in merged_entities if entity in e or e in entity):
            merged_entities.add(entity)
    return merged_entities

In [73]:
def normalize_location(loc):
    articles = {'der', 'die', 'das', 'den'}
    words = loc.split()
    normalized_words = [word for word in words if word.lower() not in articles]
    return ' '.join(normalized_words)

In [6]:
def extract_locations(text):
    doc = nlp1(text)
    locations = [ent.text for ent in doc.ents if ent.label_ == 'LOC']
    return locations

In [61]:
def extract_locations(text, known_locations):
    doc = nlp1(text)
    # Define locations based on SpaCy's entity recognition
    extracted_locations = [ent.text for ent in doc.ents if ent.label_ == 'LOC']
    # Filter the locations based on your known locations set
    locations = [loc for loc in extracted_locations if loc in known_locations]
    return locations


In [67]:
def extract_locations(text):
    doc = nlp1(text)
    locations = [ent.text for ent in doc.ents if ent.label_ == 'LOC']
    return locations


In [69]:
df_german['Voted_Locations_New'] = df_german['Content'].apply(lambda text: extract_locations_with_voting_system(text, known_locations))
df_german['Locations_New'] = df_german['Content'].apply(lambda text: extract_locations(text))


In [70]:
location_counts_new = df_german['Locations_New'].explode().value_counts()

In [77]:
df_location_counts = location_counts_new.reset_index()
df_location_counts.columns = ['LocationName', 'Count']

In [78]:
# Normalize the location names
df_location_counts['NormalizedLocation'] = df_location_counts['LocationName'].apply(normalize_location)

# Group by the normalized names and sum the counts
grouped_df_location_counts = df_location_counts.groupby('NormalizedLocation').agg({'Count': 'sum'}).reset_index()

# Sort the DataFrame by the counts in descending order
sorted_grouped_df_location_counts = grouped_df_location_counts.sort_values(by='Count', ascending=False)

In [84]:
locations_over_10_mentions = sorted_grouped_df_location_counts[sorted_grouped_df_location_counts['Count'] > 10]


In [86]:
locations_list = locations_over_10_mentions['NormalizedLocation'].tolist()

In [88]:
terms_to_remove = ['Tiktok', 'Bund', 'US-Präsident', 'Problem', 'Instagram', 'Bundesgericht', 'Ki', 'Bewohner', 'Züri Fäscht', 'Gesamt-Skigebiet', 'Hause', 'Bildstrecke']

# Remove the unwanted terms from the locations list
filtered_locations = [loc for loc in locations_list if loc not in terms_to_remove]


In [92]:
known_locations.update(filtered_locations)

In [96]:
def extract_locations_with_voting_system(text, known_locations):
    # Convert text to lowercase
    text = text.lower()
    
    # Process the lowercased text with all three models
    doc1 = nlp1(text)
    doc2 = nlp2(text)
    doc3 = nlp3(text)
    
    # Extract locations from each model and convert them to lowercase
    locations1 = {ent.text.lower() for ent in doc1.ents if ent.label_ == 'LOC'}
    locations2 = {ent.text.lower() for ent in doc2.ents if ent.label_ == 'LOC'}
    locations3 = {ent.text.lower() for ent in doc3.ents if ent.label_ == 'LOC'}
    
    # Perform a majority vote
    all_locations = locations1 | locations2 | locations3
    final_locations = {loc for loc in all_locations if sum([loc in locations for locations in [locations1, locations2, locations3]]) >= 2}
    
    # Ensure known_locations is also lowercase
    lower_known_locations = {loc.lower() for loc in known_locations}
    
    # Filter final locations to include only those in the lowercase known locations
    final_known_locations = {loc for loc in final_locations if loc in lower_known_locations}
    
    return final_known_locations

In [100]:
lower_known_locations = {location.lower() for location in known_locations if location is not None}

In [102]:
df_german['Content'] = df_german['Content'].str.lower()

In [104]:
df_german['Voted_Locations'] = df_german['Content'].apply(lambda text: extract_locations_with_voting_system(text, lower_known_locations))


In [128]:
voted_location_counts_new = df_german['Voted_Locations'].explode().value_counts()

In [124]:
entries_to_remove = {'st.', 'bund'}

# Remove these entries from the set
voted_location_counts_new = {entry for entry in voted_location_counts_new if entry not in entries_to_remove}


In [115]:
df_voted_locations = voted_location_counts_new.reset_index()
df_voted_locations.columns = ['LocationName', 'Count']

In [143]:
entries_to_remove = {'st.', 'tiktok', 'us-präsident','problem', 'bund', 'hause', 'land', 'erde', 'aa', 'instagram', 'bundesgericht', 'ki', 'bewohner', 'züri fäscht', 'gesamt-skigebiet', 'bildstrecke'}
df_voted_locations = df_voted_locations[~df_voted_locations['LocationName'].isin(entries_to_remove)]


In [169]:
file_path = 'data/occuring_locations_german.csv'
df_voted_locations.to_csv(file_path, index=False)

In [185]:
def geocode_location_positionstack(location_name, api_key):
    base_url = "http://api.positionstack.com/v1/forward"
    params = {
        'access_key': api_key,
        'query': location_name,
        'limit': 1,  # you may adjust the limit as needed
    }
    response = requests.get(base_url, params=params)
    
    if response.status_code == 200:
        data = response.json()
        # Check if data['data'] is not empty to avoid index errors
        if data['data']:
            latitude = data['data'][0]['latitude']
            longitude = data['data'][0]['longitude']
            return latitude, longitude
        else:
            return None, None
    else:
        print(f"Error: {response.status_code}")
        return None, None


In [175]:
df_filtered = df_voted_locations[df_voted_locations['Count'] >= 3].copy()



In [ ]:
geocode_api_key = '1983e85e9a97673a09ed6d19417dda0f'

df_filtered['Coordinates'] = df_filtered['LocationName'].apply(
    lambda loc: geocode_location_positionstack(loc, geocode_api_key)
)


In [224]:
df_filtered['Coordinates'] = df_filtered['Coordinates'].apply(
    lambda x: ast.literal_eval(x) if isinstance(x, str) else x
)

In [227]:
file_path = 'data/df_filtered.csv'

df_filtered.to_csv(file_path, index=False)
